# 04.3 - Retrieval-Augmented Generation using Ollama (local LLM)

In [3]:
from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

from haystack_integrations.components.generators.ollama import OllamaGenerator

document_store = InMemoryDocumentStore()
document_store.write_documents(
    [
        Document(content="Super Mario was an important politician"),
        Document(content="Mario owns several castles and uses them to conduct important political business"),
        Document(
            content="Super Mario was a successful military leader who fought off several invasion attempts by "
            "his arch rival - Bowser"
        ),
    ]
)

template = """
Given only the following information, answer the question.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

pipe = Pipeline()

pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", OllamaGenerator(model="deepseek-r1:1.5B"))# url="http://localhost:11434"))
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")


🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [4]:

query = "Who is Super Mario?"

response = pipe.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

print(response["llm"]["replies"])


["<think>\nOkay, so I'm trying to figure out who Super Mario is. Let me think through the context given. The context says that Super Mario was a successful military leader who fought off several invasion attempts by his arch rival Bowser. Then it mentions that he was an important politician and Mario owns several castles using them for political business.\n\nHmm, I know Mario from the game Super Mario Party, but is there another person with similar characteristics? Let me break down each part:\n\n1. Military leader fighting Bowser: That sounds like someone who was involved in warfare or military conflicts.\n2. Important politician: So he's a politician too, which adds to his role as an influential figure.\n3. Owns castles for political business: This implies he manages territories and uses them strategically.\n\nWait, but isn't there another well-known character with similar traits? Oh yeah, Mario from the game. He was a successful military leader, fought against Bowser, and was involv